In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


/home/venkatesh/anaconda3/envs/venkatesh_pytorch_updated_110/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_name='mobilenetv2'


if(model_name=='mobilenetv2'):
    from models.mobilenet import mobilenetv2
    model=mobilenetv2(num_classes=3)
if(model_name=='ResNet50'):
    from models.resnet import ResNet50
    model=ResNet50(num_classes=3, channels=3)    
if(model_name=='ShuffleNet2'):
    from models.ShuffleNet2 import ShuffleNet2
    model=ShuffleNet2(num_classes=3)    
if(model_name=='SqueezeNet'):
    from models.SqueezeNet import SqueezeNet
    model=SqueezeNet(num_classes=3).float().cuda()

    


In [3]:
print(model)

MobileNetV2(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
       

In [4]:
model=model.to(device)

# making directory for saving models


In [5]:
from datetime import datetime
import time

import os
print ('*******************************************************')
start_time=time.time()
experiments_folder="saved_models/"
experiment_name=datetime.now().strftime("%b_%d_%I_%M_%P_")+"_model_"+model_name

directory=experiments_folder+"/"+experiment_name+"/"
print(directory)
print('Model will be saved to  :', directory)

try:
    os.makedirs(directory)
except:
    print('model already existed..........')


*******************************************************
saved_models//Jul_29_11_59_am__model_mobilenetv2/
Model will be saved to  : saved_models//Jul_29_11_59_am__model_mobilenetv2/


In [6]:
#from models.resnet import ResNet50
#model=ResNet50(num_classes=3, channels=3)
#model=model.to(device)

In [7]:
import pandas as pd
print('traing data')
data = pd.read_csv('./csv_files/data_train.csv')

print(data.head())
# Print the statastis of the given data
print("stastics of the given data:")
print(data['label'].value_counts())
print('#'*100)

print('validation data')

data = pd.read_csv('./csv_files/data_valid.csv')

print(data.head())
# Print the statastis of the given data
print("stastics of the given data:")
print(data['label'].value_counts())
print('#'*100)

print('test data')

data = pd.read_csv('./csv_files/data_test.csv')

print(data.head())
# Print the statastis of the given data
print("stastics of the given data:")
print(data['label'].value_counts())
print('#'*100)


traing data
   Unnamed: 0                                         image_path  \
0        5758  ./processed_data/pocus_videos_to_images/convex...   
1        4244  ./processed_data/pocus_videos_to_images/convex...   
2        9517  ./processed_data/pocus_videos_to_images/convex...   
3         415  ./processed_data/pocus_videos_to_images/linear...   
4       12024  ./processed_data/pocus_videos_to_images/convex...   

                 label  
0                COVID  
1              healthy  
2                COVID  
3              healthy  
4  bacterial pneumonia  
stastics of the given data:
COVID                  3919
healthy                3697
bacterial pneumonia    2733
viral pneumonia         131
Name: label, dtype: int64
####################################################################################################
validation data
   Unnamed: 0                                         image_path  \
0       11813  ./processed_data/pocus_videos_to_images/convex...   
1        9

# Data loader

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import numpy as np


# Custom Dataset class for classification
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file, usecols=['image_path', 'label'])
        self.transform = transform
        self.class_to_int = {"COVID": 0, "healthy": 1, "bacterial pneumonia": 2,"viral pneumonia": 2}  # Mapping class names to integers



    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = self.data['image_path'][idx]
        label = self.class_to_int[self.data['label'][idx]]  # Convert class label to integer

        # Load image using PIL
        image = Image.open(image_path)

        # Convert PIL Image to NumPy array
        #image = np.array(image)
        #image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if self.transform:
            image = self.transform(image)
        #print(image.shape,label)




        image=image[:3,:,:]



        #print(np.unique(image))
        return image, label

# Transform

In [9]:
import torchvision.transforms as transforms

# Define transformations for data augmentation or normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to required size
    transforms.ToTensor()
])

In [10]:
!pip install tqdm

In [11]:

# Assuming you have a CSV file named 'data.csv' with image paths and class labels
train_csv_file = './drive/MyDrive/Aster/Aster_AI_Course/TA_Projects_Session_2_LUNG_USG_Classification/csv_files/data_train.csv'

# Create CustomDataset instance
dataset = CustomDataset('./csv_files/data_train.csv', transform=transform)
train_loader = DataLoader(dataset, batch_size=8, shuffle=False)

val_dataset = CustomDataset('./csv_files/data_valid.csv', transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

test_dataset = CustomDataset('./csv_files/data_test.csv', transform=transform)
test_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

print(len(test_dataloader))
print(len(train_loader))
print(len(val_dataloader))
print(len(test_dataloader))






164
1310
164
164


In [12]:
num_classes = 3  # Assuming 3 classes: "covid", "normal", "pneumonia"


learning_rate=0.0001
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train your model
from tqdm import tqdm

# Define your training function
model.train()
train_losses = []  # List to store training losses
val_losses = []  # List to store validaion losses

num_epochs=25



for epoch in range(num_epochs):
    running_loss = 0.0
    validation_loss=0.0
    
    y_true_train = []
    y_pred_train = []


    # Create tqdm progress bar
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}')

    for batch_idx, (images, labels) in progress_bar:
        images = images.to(device)
        labels = labels.to(device)
        #images=images/255
        #print(torch.unique(images))
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        y_pred_train.extend(outputs.detach().argmax(dim=-1).tolist())
        y_true_train.extend(labels.detach().tolist())

        
        # Update progress bar description with current loss
        progress_bar.set_postfix({'Loss': running_loss / (batch_idx + 1)})

        
    total_correct = len([True for x, y in zip(y_pred_train, y_true_train) if x==y])
    total = len(y_pred_train)
    accuracy = total_correct * 100 / total
    print("-------------------------------------------------")
    print("Epoch: {} Train mean loss: {:.8f}".format(epoch, running_loss))
    print("Train Accuracy%: ", accuracy, "==", total_correct, "/", total)
    print("-------------------------------------------------")
        
    save_path=directory+'/mymodel_'+str(epoch)+'.pth'
    torch.save(model.state_dict(), save_path)
    print(f'Model saved at: {save_path}')

    # storing the train losses
    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    progress_bar = tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc=f'Epoch {epoch + 1}/{num_epochs}')

    y_true_val = []
    y_pred_val = []

    
    model.eval()
    for batch_idx, (images, labels) in progress_bar:
        images = images.to(device)
        labels = labels.to(device)
        #images=images/255
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        validation_loss += loss.item()

        y_pred_val.extend(outputs.detach().argmax(dim=-1).tolist())
        y_true_val.extend(labels.detach().tolist())

        
        
        # Update progress bar description with current loss
        progress_bar.set_postfix({'Validation Loss': validation_loss / (batch_idx + 1)})

    # storing the validation losses

    epoch_loss = validation_loss / len(val_dataloader)
    val_losses.append(epoch_loss)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    total_correct = len([True for x, y in zip(y_pred_val, y_true_val) if x==y])
    total = len(y_true_val)
    accuracy = total_correct * 100 / total

    print("-------------------------------------------------")
    print("Epoch: {} Val mean loss: {:.8f}".format(epoch, validation_loss))
    print("valiation Accuracy%: ", accuracy, "==", total_correct, "/", total)
    print("-------------------------------------------------")


Epoch 1/25: 100%|██████████| 1310/1310 [01:28<00:00, 14.82it/s, Loss=0.376]


-------------------------------------------------
Epoch: 0 Train mean loss: 492.27969976
Train Accuracy%:  84.74236641221374 == 8881 / 10480
-------------------------------------------------
Model saved at: saved_models//Jul_29_11_59_am__model_mobilenetv2//mymodel_0.pth
Epoch [1/25], Loss: 0.3758


Epoch 1/25: 100%|██████████| 164/164 [00:08<00:00, 20.18it/s, Validation Loss=0.0945]


Epoch [1/25], Loss: 0.0945
-------------------------------------------------
Epoch: 0 Val mean loss: 15.49989851
valiation Accuracy%:  97.32824427480917 == 1275 / 1310
-------------------------------------------------


Epoch 2/25: 100%|██████████| 1310/1310 [01:25<00:00, 15.29it/s, Loss=0.113]


...

Epoch 15/25: 100%|██████████| 1310/1310 [01:26<00:00, 15.10it/s, Loss=0.00779]


-------------------------------------------------
Epoch: 14 Train mean loss: 10.20162434
Train Accuracy%:  99.78053435114504 == 10457 / 10480
-------------------------------------------------
Model saved at: saved_models//Jul_29_11_59_am__model_mobilenetv2//mymodel_14.pth
Epoch [15/25], Loss: 0.0078


Epoch 15/25: 100%|██████████| 164/164 [00:08<00:00, 19.78it/s, Validation Loss=0.0134] 


Epoch [15/25], Loss: 0.0134
-------------------------------------------------
Epoch: 14 Val mean loss: 2.20502418
valiation Accuracy%:  99.8473282442748 == 1308 / 1310
-------------------------------------------------


Epoch 16/25:   7%|▋         | 90/1310 [00:06<01:19, 15.26it/s, Loss=0.00279] 

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation losses
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
plt.show()

# Model Analysis

# Total params: 2,227,715

Trainable params: 2,227,715

Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57

Forward/backward pass size (MB): 152.86

Params size (MB): 8.50

Estimated Total Size (MB): 161.93

Computational complexity: 911.67 MMac


Computational complexity: 1823.34 MFlops

Number of parameters: 2.23 M 

In [ ]:
from torchsummary import summary

summary(model, (3, 224, 224))

In [ ]:
import re
from ptflops import get_model_complexity_info
from pthflops import count_ops

macs, params = get_model_complexity_info(model, (3, 448, 320), as_strings=True,print_per_layer_stat=False, verbose=False)
# # Extract the numerical value
flops = eval(re.findall(r'([\d.]+)', macs)[0])*2
# Extract the unit
flops_unit = re.findall(r'([A-Za-z]+)', macs)[0][0]

print('Computational complexity: {:<8}'.format(macs))
print('Computational complexity: {} {}Flops'.format(flops, flops_unit))
print('Number of parameters: {:<8}'.format(params))


# Testing 

In [ ]:
# choosing the best model from validation loss.

best_epoch=np.argmin(val_losses)
print('best_epoch:',best_epoch)

In [ ]:
ckpt = torch.load('./saved_models/mymodel_'+str(best_epoch)+'.pth')
model.load_state_dict(ckpt)
model.eval()


In [ ]:
from tqdm import tqdm

progress_bar = tqdm(enumerate(test_dataloader), total=len(test_dataloader), desc=f'testing')

y_true_val = []
y_pred_val = []


model.eval()
for batch_idx, (images, labels) in progress_bar:
    images = images.to(device)
    labels = labels.to(device)
    #images=images/255
    optimizer.zero_grad()

    outputs = model(images)

    y_pred_val.extend(outputs.detach().argmax(dim=-1).tolist())
    y_true_val.extend(labels.detach().tolist())



    # Update progress bar description with current loss



total_correct = len([True for x, y in zip(y_pred_val, y_true_val) if x==y])
total = len(y_true_val)
accuracy = total_correct * 100 / total

print("-------------------------------------------------")
print("TEST Accuracy (%): ", accuracy, "==", total_correct, "/", total)
print("-------------------------------------------------")
